## Stochastic environment (First requirement)

In [1]:
import numpy as np
import matplotlib.pyplot as plt 

### Stochastic environment 
The stochastic environment takes as "input" the action of the agent which corresponds to the bid. Then it simulates the distribution from which the other bids are sampled and the probability conversion (prob. of buying given a price -> demand curve normalized). Eventually, it outputs the reward associated with  

In [ ]:
class stochastic_env():
    def __init__(probability_conversion):
        #self.rewards 
        self.probability_conversion = probability_conversion

    def round():
        # 
        pass

    

### Pricing algorithm
Build a pricing strategy using the continuous set of prices $p \in [0,1]$ and Gaussian Processes

In [2]:
# taken from lab session 3
class RBFGaussianProcess:
    def __init__(self, scale=1, reg=1e-2):
        self.scale = scale 
        self.reg = reg
        self.k_xx_inv = None

    def rbf_kernel_incr_inv(self, B, C, D):
        temp = np.linalg.inv(D - C @ self.k_xx_inv @ B)
        block1 = self.k_xx_inv + self.k_xx_inv @ B @ temp @ C @ self.k_xx_inv
        block2 = - self.k_xx_inv @ B @ temp
        block3 = - temp @ C @ self.k_xx_inv
        block4 = temp
        res1 = np.concatenate((block1, block2), axis=1)
        res2 = np.concatenate((block3, block4), axis=1)
        res = np.concatenate((res1, res2), axis=0)
        return res

    def rbf_kernel(self, a, b):
        a_ = a.reshape(-1, 1)
        b_ = b.reshape(-1, 1)
        output = -1 * np.ones((a_.shape[0], b_.shape[0]))
        for i in range(a_.shape[0]):
            output[i, :] = np.power(a_[i] - b_, 2).ravel()
        return np.exp(-self.scale * output)
    
    def fit(self, x=np.array([]), y=np.array([])):
        x,y = np.array(x),np.array(y)
        if self.k_xx_inv is None:
            self.y = y.reshape(-1,1)
            self.x = x.reshape(-1,1)
            k_xx = self.rbf_kernel(self.x, self.x) + self.reg * np.eye(self.x.shape[0])
            self.k_xx_inv = np.linalg.inv(k_xx)
        else:
            B = self.rbf_kernel(self.x, x)
            self.x = np.vstack((self.x, x))
            self.y = np.vstack((self.y, y))
            self.k_xx_inv = self.rbf_kernel_incr_inv(B, B.T, np.array([1 + self.reg]))

        return self

    def predict(self, x_predict):
        k = self.rbf_kernel(x_predict, self.x)

        mu_hat = k @ self.k_xx_inv @ self.y
        sigma_hat = 1 - np.diag(k @ self.k_xx_inv @ k.T)

        return mu_hat.ravel(), sigma_hat.ravel()

Implementing the Gaussian Processes in our agent:

In [3]:
# taken from lab session 3 as well
class pricing_agent():
    def __init__(self, T, discretization=100):
        self.T = T
        self.arms = np.linspace(0, 1, discretization)
        self.gp = RBFGaussianProcess(scale=2).fit()
        self.a_t = None
        self.action_hist = np.array([])
        self.reward_hist = np.array([])
        self.mu_t = np.zeros(discretization)
        self.sigma_t = np.zeros(discretization)
        self.gamma = lambda t: np.log(t+1)**2 
        self.beta = lambda t: 1 + 0.5*np.sqrt(2 * (self.gamma(t) + 1 + np.log(T)))      # from PAPER LINKED IGP-UCB ALGORITHM
        self.N_pulls = np.zeros(discretization)
        self.t = 0
    
    def pull_arm(self):
        self.mu_t, self.sigma_t = self.gp.predict(self.arms) 
        ucbs = self.mu_t + self.beta(t) * self.sigma_t
        self.a_t = np.argmax(ucbs)
        return self.arms[self.a_t]
    
    def update(self, r_t):
        self.N_pulls[self.a_t] += 1
        self.action_hist = np.append(self.action_hist, self.arms[self.a_t])
        self.reward_hist = np.append(self.reward_hist, r_t)
        self.gp = self.gp.fit(self.arms[self.a_t], r_t)
        self.t += 1

### Bidding algorithms
Consider a sequence of second-price auctions. Build two learning algorithms to deal with the bidding problem:
- a primal-dual algorithm for truthful auctions
- a UCB-like algorithm

Since we are in a "second-price case" this means that we are in non-truthful auctions. 

In [ ]:
class primal_dual_agent():
    def __init__(self) -> None:
        pass 
    
    def action():
        # choose price
        pass
    
    def update():
        pass

This can be implemented following the algorithm in the slide 18 from slide package 8.

In [ ]:
class UCB_agent():
    def __init__(self) -> None:
        pass 
    
    def action():
        # choose price
        pass
    
    def update():
        pass

### General framework to be modeled
Doubts: 
- how do we define how lambda changes depending on s (i.e. the probability of seeing the ad given its slot position)
- is there an auction for each slot or they are associated depending on how much money are spent for just one auction

In [5]:
ITERATIONS = 1000
AUCTIONS = 100
N_CUSTOMERS = 100
N_COMPANIES = 10    # number of companies that compete for slot auctions
LAMBDA = lambda s: 1/s+1       # probability of the ad being seen given the position s ->  0.5 for first position, 0.33 for second, 0.25 for third, etc...

pricing_agent = pricing_agent(ITERATIONS, 100)

for round in range(ITERATIONS):
    pricing_agent.pull_arm()    # pricing agent choose a price p for the product

    for auction in range(AUCTIONS):
        # bidding agent decides how much to bid 
        # somebody wins the auction
        # the environment decides how to associate the probability of click given the position of the ad <- FIXED
        # the product might be bought depending on the demand curve <- defined in the environment
        # the agent receives a reward
        pass